In [ ]:
import pandas as pd
loaded_df = pd.read_pickle("....pkl")

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, peak_widths
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.special import gamma
from scipy.special import voigt_profile


# Old Method

In [ ]:


i1=5
i2=14
input_data_size = len(loaded_df)
new_training_data = np.zeros((input_data_size * i1 * i2, 2251))
label_lists = []

np.random.seed(1)




for i in range(input_data_size):
    #print(i)
    label = loaded_df.iloc[i, 0]
    print(label)
    theoretical = loaded_df.iloc[i, 1:].values
    len1 = len(theoretical)

    lc_id1 = i * i1 * i2

    for j in range(i1):
        lc_id2 = j * i2

        for k in range(i2):  # expand each data to 6*12=72 data
            copy_theo = np.copy(theoretical)  # Create a copy of the original data to avoid modifying it directly

            # Replace the peak-specific operations with the random elimination and scaling processes
            # Random elimination
            dum1 = np.repeat(np.random.choice([0, 1], 300, p=[0.2, 0.8]), len(copy_theo) // 300)
            dum1 = np.append(dum1, np.zeros([len(copy_theo) - len(dum1), ]))
            copy_theo = np.multiply(copy_theo, dum1)

            # Random scaling
            dum2 = np.repeat(np.random.rand(150,), len(copy_theo) // 150)
            dum2 = np.append(dum2, np.zeros([len(copy_theo) - len(dum2), ]))
            copy_theo = np.multiply(copy_theo, dum2)


    
            # Normalize the data
            copy_theo_elimination_scaling=copy_theo
            copy_theo_elimination_scaling = (copy_theo_elimination_scaling - np.min(copy_theo_elimination_scaling)) / (
                np.max(copy_theo_elimination_scaling) - np.min(copy_theo_elimination_scaling) + 1e-9)

            # Add noise
            #noise_intensity = np.random.uniform(0, 0.005)
            noise_intensity=0
            noise = np.random.normal(0, noise_intensity * np.max(copy_theo_elimination_scaling), len(copy_theo_elimination_scaling))
            copy_theo_elimination_scaling += noise

            # Normalize again
            copy_theo_elimination_scaling = (copy_theo_elimination_scaling - np.min(copy_theo_elimination_scaling)) / (
                np.max(copy_theo_elimination_scaling) - np.min(copy_theo_elimination_scaling) + 1e-9)

            # Left-right shifting process
            shift = np.random.randint(-25 * 1, 25)  # Generate a random integer between -20 and 20 for left-right shifting
            if shift >= 0:
                # Shift data to the right
                copy_theo_elimination_scaling = np.append(copy_theo_elimination_scaling[shift:], np.zeros([shift,]))
            else:
                # Shift data to the left
                copy_theo_elimination_scaling = np.append(copy_theo_elimination_scaling[0:len1 + shift],
                                                          np.zeros([shift * -1,]))


            lc_id = lc_id1 + lc_id2 + k

            new_training_data[lc_id] = copy_theo_elimination_scaling
            label_lists.append(label)  # Move the label appending inside the loop

new_train = new_training_data.tolist()

# Create a new dataframe for the processed data
new_train_df = pd.DataFrame(new_train)

# Add labels as the first column
new_train_df.insert(0, 'Label', label_lists)

print('the total train data number is:  {}'.format(len(new_train)))
print('the total train label number is:  {}'.format(len(label_lists)))

# Save the new dataframe to a file
#new_train_df.to_csv('augmented_data.csv', index=False)



# Our Method

In [ ]:


i1=10
i2=20
input_data_size = len(loaded_df)
new_training_data = np.zeros((input_data_size * i1 * i2, 2251))
label_lists = []

np.random.seed(1)




for i in range(input_data_size):
    #print(i)
    label = loaded_df.iloc[i, 0]
    print(label)
    theoretical = loaded_df.iloc[i, 1:].values
    len1 = len(theoretical)

    lc_id1 = i * i1 * i2

    for j in range(i1):
        lc_id2 = j * i2

        for k in range(i2):  # expand each data to 6*12=72 data
            copy_theo = np.copy(theoretical)  # Create a copy of the original data to avoid modifying it directly

            # Replace the peak-specific operations with the random elimination and scaling processes
            # Random elimination
            dum1 = []
            while len(dum1) < len(copy_theo):
                repeat_count = np.random.randint(7, 9)  # Randomly choose a number between 7 and 8
                dum1 = np.append(dum1, np.repeat(np.random.choice([0, 1], 1, p=[0.2, 0.8]), repeat_count))
            dum1 = dum1[:len(copy_theo)]  # if dum1 is longer than copy_theo, truncate it
            copy_theo = np.multiply(copy_theo, dum1)
            
            dum2 = []
            while len(dum2) < len(copy_theo):
                dum2 = np.repeat(np.random.rand(150,), len(copy_theo) // 150)
                repeat_count = np.random.randint(14, 17)  # Randomly choose a number between 7 and 8
                dum2 = np.append(dum2, np.repeat(np.random.rand(150,), repeat_count))
            dum2 = dum2[:len(copy_theo)]  # if dum1 is longer than copy_theo, truncate it
            copy_theo = np.multiply(copy_theo, dum2)            
        


    
            # Normalize the data
            copy_theo_elimination_scaling=copy_theo
            copy_theo_elimination_scaling = (copy_theo_elimination_scaling - np.min(copy_theo_elimination_scaling)) / (
                np.max(copy_theo_elimination_scaling) - np.min(copy_theo_elimination_scaling) + 1e-9)

            # Add noise
            #noise_intensity = np.random.uniform(0, 0.005)
            noise_intensity=0
            noise = np.random.normal(0, noise_intensity * np.max(copy_theo_elimination_scaling), len(copy_theo_elimination_scaling))
            copy_theo_elimination_scaling += noise

            # Normalize again
            copy_theo_elimination_scaling = (copy_theo_elimination_scaling - np.min(copy_theo_elimination_scaling)) / (
                np.max(copy_theo_elimination_scaling) - np.min(copy_theo_elimination_scaling) + 1e-9)

            # Left-right shifting process
            shift = np.random.randint(-25 * 1, 25)  # Generate a random integer between -20 and 20 for left-right shifting
            if shift >= 0:
                # Shift data to the right
                copy_theo_elimination_scaling = np.append(copy_theo_elimination_scaling[shift:], np.zeros([shift,]))
            else:
                # Shift data to the left
                copy_theo_elimination_scaling = np.append(copy_theo_elimination_scaling[0:len1 + shift],
                                                          np.zeros([shift * -1,]))


            lc_id = lc_id1 + lc_id2 + k

            new_training_data[lc_id] = copy_theo_elimination_scaling
            label_lists.append(label)  # Move the label appending inside the loop

#new_train = new_training_data.tolist()

# Create a new dataframe for the processed data
#new_train_df = pd.DataFrame(new_train)
new_train_df = pd.DataFrame(new_training_data)

# Add labels as the first column
new_train_df.insert(0, 'Label', label_lists)

print('the total train data number is:  {}'.format(len(new_train)))
print('the total train label number is:  {}'.format(len(label_lists)))

# Save the new dataframe to a file
#new_train_df.to_csv('augmented_data.csv', index=False)



In [ ]:
import pandas as pd
new_train_df.to_pickle("....pkl")